In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mplsoccer import Pitch
from sklearn.cluster import KMeans

import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('D:\Futbol\Analisis de datos\Analisis presion alata en PL 23-24\data\pl_23-24_events.csv')

In [3]:
df['distance'] = np.sqrt(((105/100) * (df['endX'] - df['x']))**2 + ((68/100) * (df['endY'] - df['y']))**2)
df['angle'] = np.degrees(np.arctan2(df['endY'] - df['y'], df['endX'] - df['x']))

In [4]:
df_pass = df[(df['teamId'] == 30) & (df['type'] == 'Pass') & (df['matchId'] == 1729441)]

In [5]:
df_pp_1 = df_pass[(df_pass['distance'] >= 30) & (df_pass['passForward'] == True) & (df['x'] <= 50)]
df_pp_2 = df_pass[(df_pass['distance'] >= 15) & (df_pass['passForward'] == True) & (df['x'] <= 50) & (df['endX'] >= 50)]
df_pp_3 = df_pass[(df_pass['distance'] >= 10) & (df['x'] >= 50)]
df_pp = pd.concat([df_pp_1, df_pp_2, df_pp_3])

In [31]:
K = np.linspace(1, 20, 20)
X = united_progressive[["x", "y", "end_x", "end_y", "angle"]].values
elbow = {"sse": [], "k": [], "sil": []}
for k in K:
    cluster = KMeans(n_clusters = int(k), random_state = 2147)
    labels = cluster.fit_predict(X)
    elbow["sse"].append(cluster.inertia_)
    elbow["k"].append(k)

plt.scatter(elbow["k"], elbow["sse"])
plt.plot(elbow["k"], elbow["sse"])
plt.xticks(np.linspace(1, 20, 20))
plt.xlabel("K")
plt.ylabel("SSE")
plt.show()
from sklearn.metrics import pairwise_distances
def compute_inertia(a, X):
    W = [np.mean(pairwise_distances(X[a == c, :])) for c in np.unique(a)]
    return np.mean(W)

def compute_gap(clustering, data, k_max, n_references=5):
    if len(data.shape) == 1:
        data = data.reshape(-1, 1)
    reference = np.random.rand(*data.shape)
    reference_inertia = []
    for k in range(1, k_max+1):
        local_inertia = []
        for _ in range(n_references):
            clustering.n_clusters = k
            assignments = clustering.fit_predict(reference)
            local_inertia.append(compute_inertia(assignments, reference))
        reference_inertia.append(np.mean(local_inertia))

    ondata_inertia = []
    for k in range(1, k_max+1):
        clustering.n_clusters = k
        assignments = clustering.fit_predict(data)
        ondata_inertia.append(compute_inertia(assignments, data))

    gap = np.log(reference_inertia)-np.log(ondata_inertia)
    return gap, np.log(reference_inertia), np.log(ondata_inertia)

k_max = 10
gap, reference_inertia, ondata_inertia = compute_gap(KMeans(random_state = 2147), X, k_max)

plt.plot(range(1, k_max+1), gap, '-o')
plt.ylabel('gap')
plt.xlabel('k')

,Unnamed: 0,id,eventId,minute,second,teamId,h_a,x,y,expandedMinute,...,subOn,subOff,defensiveThird,midThird,finalThird,pos,EPV,isOwnGoal,distancia_metros,distancia
586401,139,2.687416e+09,84,5,16.0,30,h,32.0,79.1,5,...,False,False,False,False,True,True,0.0133,NaN,61.970474,61.970474
586420,158,2.687417e+09,98,6,51.0,30,h,41.6,26.2,6,...,False,False,False,False,True,True,NaN,NaN,42.740290,42.740290
586460,198,2.687418e+09,119,9,17.0,30,h,12.3,95.9,9,...,False,False,False,True,False,True,NaN,NaN,34.202602,34.202602
586486,224,2.687419e+09,134,10,26.0,30,h,10.6,41.6,10,...,False,False,False,True,False,True,NaN,NaN,46.506877,46.506877
586663,401,2.687425e+09,226,19,8.0,30,h,9.7,63.7,19,...,False,False,False,True,False,True,0.0031,NaN,34.429807,34.429807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587962,1700,2.687461e+09,934,99,48.0,30,h,59.6,92.4,100,...,False,False,False,True,False,True,0.0020,NaN,14.868539,14.868539
587963,1701,2.687461e+09,935,99,50.0,30,h,57.4,70.8,100,...,False,False,False,True,False,True,0.0032,NaN,21.211041,21.211041
587964,1702,2.687461e+09,937,99,53.0,30,h,69.1,36.9,100,...,False,False,False,False,True,True,-0.0012,NaN,19.044614,19.044614
587968,1706,2.687462e+09,939,100,0.0,30,h,57.9,24.1,101,...,False,False,False,True,False,True,0.0017,NaN,13.413087,13.413087
